# iNaturalist status updates by state - VIC

Using the file produced in the collate-status-taxa.ipynb: `inat-aust-status-taxa.csv`, generate lists to update iNaturalist statuses

## Prep - common to all states
1. Read in the inaturalist statuses & filter out this state
2. Read in the inaturalist taxa list
3. Read in the state sensitive and conservation list
4. Attempt to match the state statuses to an IUCN equivalent
5. Determine the best placeID to use for this state

**Next steps:**
Establish the changes that need to be made. Read in the sensitive and conservation list for each state.
    a. new - any new species that appear in the state lists but do not have a status in inaturalist (new template)
    b. updates - any changes to information which was added by us previously (user_id = 708886) (update template, action='UPDATE')
    c. removals - any statuses which were added by us previously (user_id = 708886) list which are incorrect (update template, action='REMOVE')
    d. flags - are there any statuses by other users that need to be flagged?

### 1. iNaturalist statuses

In [74]:
import pandas as pd

projectdir = "/Users/oco115/PycharmProjects/authoritative-lists/" # basedir for this gh project
# projectdir = "/Users/new330/IdeaProjects/authoritative-lists/" # basedir for this gh project
sourcedir = projectdir + "source-data/inaturalist-statuses/"
listdir = projectdir + "current-lists/"

# read in the statuses
taxastatus = pd.read_csv(sourcedir + "inat-aust-status-taxa.csv", encoding='UTF-8',na_filter=False,dtype=str) ## Read inaturalist conservation statuses file
taxastatus.head(3)


,id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
0,166449,38493,1138587,7830,,Flora and Fauna Guarantee Act 1988,CR,,,obscured,...,Eulamprus,kosciuskoi,,2021-03-01T10:35:01Z,Eulamprus kosciuskoi,species,http://reptile-database.reptarium.cz/search.ph...,,,
1,234788,918383,702203,9994,,Atlas of Living Australia,NT,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,
2,234789,918383,702203,7308,,Atlas of Living Australia,LC,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,


In [75]:
def filter_state_statuses(stateregex: str, urlregex: str):
    authoritydf = taxastatus['authority'].drop_duplicates().sort_values()
    authoritydf = authoritydf[pd.Series(authoritydf).str.contains(stateregex)]
    urldf = taxastatus['url'].drop_duplicates().sort_values()
    urldf = urldf[pd.Series(urldf).str.contains(urlregex)]
    placedisplaydf = taxastatus['place_display_name'].drop_duplicates().sort_values()
    placedisplaydf = placedisplaydf[pd.Series(placedisplaydf).str.contains(stateregex)]
    placedf = taxastatus['place_name'].drop_duplicates().sort_values()
    placedf = placedf[pd.Series(placedf).str.contains(stateregex)]
    # concat all and remove duplicates
    statedf = pd.concat([taxastatus.apply(lambda row: row[taxastatus['place_display_name'].isin(placedisplaydf)]),
                         taxastatus.apply(lambda row: row[taxastatus['place_name'].isin(placedf)]),
                         taxastatus.apply(lambda row: row[taxastatus['url'].isin(urldf)]),
                         taxastatus.apply(
                             lambda row: row[taxastatus['authority'].isin(authoritydf)])]).drop_duplicates()
    return statedf.sort_values(['taxon_id', 'user_id'])

inatstatuses = filter_state_statuses(" SA |South Australia|SOUTH AUSTRALIA","sa.gov.au")
inatstatuses.rename(columns={'id':'status_id','id_y':'taxon_id_y'},inplace=True)
inatstatuses

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
147,152333,100616,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Euastacus,bispinosus,,2022-06-06T16:26:39Z,Euastacus bispinosus,species,http://www.iucnredlist.org/apps/redlist/details,,,
89,180750,1038978,624851,6899,,"Department of Environment and Water, Governmen...",Endangered,,Known from just a few areas and near human dis...,obscured,...,Hibbertia,hirsuta,,2021-08-31T12:37:27Z,Hibbertia hirsuta,species,,,,
2631,163879,1094266,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Lagenophora,sublyrata,,2020-11-16T20:50:54Z,Lagenophora sublyrata,species,,,,
389,223419,1104996,624851,6899,,"Department of Environment and Water, Governmen...",E,,Endangered in South Australia because it known...,obscured,...,Ptilotus,angustifolius,,2021-10-17T07:32:41Z,Ptilotus angustifolius,species,https://eol.org/pages/55577306,,,
922,152427,116999,,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,,...,Pandion,haliaetus,,2022-06-27T03:09:49Z,Pandion haliaetus,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,170156,892980,702203,6899,,"Department of Environment and Heritage, South ...",Threatened,https://www.landscape.sa.gov.au › threatened_p...,,,...,Daviesia,pectinata,,2021-07-27T12:40:15Z,Daviesia pectinata,species,http://www.catalogueoflife.org/annual-checklis...,,,
1257,152426,892983,708886,6899,16651,SA DEWNR,Endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Caladenia,lowanensis,,2022-10-22T05:13:46Z,Caladenia lowanensis,species,http://www.catalogueoflife.org/annual-checklis...,,,
468,152359,894302,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Caladenia,fragrantissima,,2022-06-07T15:06:33Z,Caladenia fragrantissima,species,http://www.catalogueoflife.org/annual-checklis...,,,
245,170258,950220,702203,6899,,Atlas of Living Australia,EN,https://bie.ala.org.au/species/https://id.biod...,,obscured,...,Caladenia,colorata,,2022-06-09T14:15:02Z,Caladenia colorata,species,http://www.catalogueoflife.org/annual-checklis...,,,


### 2. iNaturalist taxonomy

In [76]:
# Output files contain these fields
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# so we need to match species from the state lists to the inat taxa to get the taxon_id

import zipfile
url = "https://www.inaturalist.org/taxa/inaturalist-taxonomy.dwca.zip"
filename = url.split("/")[-1]

z=zipfile.ZipFile(sourcedir + filename)

with z.open('taxa.csv') as from_archive:
    inattaxa = pd.read_csv(from_archive,dtype=str)
z.close()
inattaxa.head(3)


,id,taxonID,identifier,parentNameUsageID,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references
0,1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/48460,Animalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-02T06:05:44Z,Animalia,kingdom,http://www.catalogueoflife.org/annual-checklis...
1,2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/1,Animalia,Chordata,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-23T00:40:18Z,Chordata,phylum,http://www.catalogueoflife.org/annual-checklis...
2,3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/355675,Animalia,Chordata,Aves,NaN,NaN,NaN,NaN,NaN,2022-12-27T07:33:16Z,Aves,class,http://www.catalogueoflife.org/annual-checklis...


### 3. State lists

Get the ALA Conservation and Sensitive lists


In [77]:
# %%script echo skipping # comment this line to download dataset from lists.ala.org.au the web and save locally
import sys
import os
sys.path.append(os.path.abspath(projectdir + "source-code/includes"))
import list_functions as lf
# no sensitive list in lists-test
sensitivelist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr884?max=10000&includeKVP=true")
sensitivelist = lf.kvp_to_columns(sensitivelist)
sensitivelist.to_csv(sourcedir + "sa-ala-sensitive.csv", index=False)

conservationlist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr653?max=10000&includeKVP=true")
conservationlist = lf.kvp_to_columns(conservationlist)
conservationlist.to_csv(sourcedir + "sa-ala-conservation.csv", index=False)

In [78]:
# Read sensitive list data
sensitivelist = pd.read_csv(sourcedir + "sa-ala-sensitive.csv", dtype=str)
sensitivelist['sa_geoprivacy'] = 'obscured'
sensitivelist['status']= 'sensitive'
sensitivelist = sensitivelist.rename(columns={'NSXCODE':'taxonID'})
sensitivelist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,Grouping,SPECIESTYPE,taxonID,vernacular name,Clarifier,Threaten a species/environment;,Threaten someones privacy;,Reason,sa_geoprivacy,status
0,24233,Tiliqua adelaidensis,Pygmy Bluetongue,Tiliqua adelaidensis,https://biodiversity.org.au/afd/taxa/a70c19ab-...,dr884,"[{'key': 'Grouping', 'value': 'Reptiles'}, {'k...",Reptiles,R,Y02572,Pygmy Blue-tongue Lizard (listed as Pygmy Blue...,All known location and burrow records,y,y,The Pygmy Blue-tongue Lizard is endemic to SA ...,obscured,sensitive
1,24377,Calyptorhynchus banksii graptogyne,Red-tailed Black Cockatoo (south-east),Calyptorhynchus (Calyptorhynchus) banksii grap...,https://biodiversity.org.au/afd/taxa/f8ecc7e3-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,Y09312,Red-tailed Black-Cockatoo (south east subspecies),All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive
2,24336,Calyptorhynchus lathami halmaturinus,Glossy Black-cockatoo (kangaroo Island Subspec...,Calyptorhynchus (Calyptorhynchus) lathami halm...,https://biodiversity.org.au/afd/taxa/d2f5121c-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,C00265,Glossy Black-Cockatoo (Kangaroo Island subspec...,All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive
3,24331,Lophochroa leadbeateri,Major Mitchell's Cockatoo,Lophochroa leadbeateri,https://biodiversity.org.au/afd/taxa/5815e99d-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,U00270,Major Mitchell's Cockatoo (listed as Cacatua l...,All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive
4,24267,Calyptorhynchus funereus,Yellow-tailed Black-cockatoo,Zanda funerea,https://biodiversity.org.au/afd/taxa/145b081d-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,G00267,Yellow-tailed Black Cockatoo,All known nest hollows and artificial nest boxes.,y,NaN,some associated subspecies records are sensitive,obscured,sensitive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,4221188,Ixodia achillaeoides subsp. arenicola,NaN,Ixodia achillaeoides subsp. arenicola,https://id.biodiversity.org.au/node/apni/2916083,dr884,"[{'key': 'Grouping', 'value': 'Garden Trade -O...",Garden Trade -Other Ornamental Horticulture pl...,P,ALA concept,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive
247,4221189,Asplenium bulbiferum subsp. gracillimum,Mother Spleenwort,Asplenium bulbiferum subsp. gracillimum,https://id.biodiversity.org.au/node/apni/2910577,dr884,"[{'key': 'Grouping', 'value': 'Ferns & Allies'...",Ferns & Allies,P,ALA concept,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,Taxon is listed as ssp. gracillimum so subsp. ...,obscured,sensitive
248,4221190,Asplenium bulbiferum,Hen And Chickens,Asplenium bulbiferum,https://id.biodiversity.org.au/node/apni/2887052,dr884,"[{'key': 'Grouping', 'value': 'Ferns & Allies'...",Ferns & Allies,P,NaN,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive
249,4221191,Lagenophora sublyrata,Slender Bottle-daisy,Lagenophora sublyrata,https://id.biodiversity.org.au/name/apni/51338482,dr884,"[{'key': 'Grouping', 'value': 'Garden Trade -O...",Garden Trade -Other Ornamental Horticulture pl...,P,na,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive


In [79]:
conservationlist = pd.read_csv(sourcedir + "sa-ala-conservation.csv", dtype=str)
conservationlist['sa_geoprivacy'] = 'open'
conservationlist['co_status'] = conservationlist['status']
conservationlist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,class,order,family,...,scientificNameAuthorship,status,npwActStatusComments,epbcStatus,epbcActStatusComments,subspeciesAuthor,majorGroup,subSpecies,sa_geoprivacy,co_status
0,4032241,Ornithorhynchus anatinus,Platypus,Ornithorhynchus anatinus,https://biodiversity.org.au/afd/taxa/ac61fd14-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,MONOTREMATA,ORNITHORHYNCHIDAE,...,"(Shaw, 1799)",Endangered,NaN,NaN,NaN,NaN,NaN,NaN,open,Endangered
1,4032333,Tachyglossus aculeatus multiaculeatus,Short-beaked Echidna (kangaroo Island),Tachyglossus aculeatus multiaculeatus,https://biodiversity.org.au/afd/taxa/e9289876-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,MONOTREMATA,TACHYGLOSSIDAE,...,"(Rothschild, 1905)",Endangered,Listing official as off 1/Jan/2020.,EN,NaN,NaN,NaN,NaN,open,Endangered
2,4031684,Myrmecobius fasciatus,Numbat,Myrmecobius fasciatus,https://biodiversity.org.au/afd/taxa/6c72d199-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,DASYUROMORPHIA,MYRMECOBIIDAE,...,"Waterhouse, 1836",Endangered,Introduced,EN,Introduced to South Australia. Changed from VU...,NaN,NaN,NaN,open,Endangered
3,4032310,Dasycercus blythi,Brush-tailed Mulgara,Dasycercus blythi,https://biodiversity.org.au/afd/taxa/926b1c33-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,DASYUROMORPHIA,DASYURIDAE,...,"(Krefft, 1867)",Endangered,NaN,NaN,NaN,NaN,NaN,NaN,open,Endangered
4,4031916,Dasyuroides byrnei,Kowari,Dasyuroides byrnei,https://biodiversity.org.au/afd/taxa/c342ff42-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,DASYUROMORPHIA,DASYURIDAE,...,"(Spencer, 1896)",Endangered,Changed from V to E 1/Jan/2020.,VU,NaN,NaN,NaN,NaN,open,Endangered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,4032775,Thelymitra matthewsii,Spiral Sun-orchid,Thelymitra matthewsii,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra matthewsii Cheeseman,Endangered,NaN,VU,2000-07-16 00:00:00,NaN,Angiospermae,NaN,open,Endangered
1168,4032248,Thelymitra occidentalis,Western Azure Sun Orchid,Thelymitra occidentalis,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra occidentalis Jeanes,Rare,RA d(i),NaN,NaN,NaN,Angiospermae,NaN,open,Rare
1169,4031900,Thelymitra orientalis,Plum Sun-orchid,Thelymitra orientalis,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra orientalis R.J.Bates,Rare,NaN,NaN,NaN,NaN,Angiospermae,NaN,open,Rare
1170,4032428,Thelymitra peniculata,Blue Star Sun-orchid,Thelymitra peniculata,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra peniculata Jeanes,Vulnerable,Warning: Conservation status data auto-copied ...,NaN,NaN,NaN,Angiospermae,NaN,open,Vulnerable


In [80]:
# Sensitive list does not have a status.
# Records need status from Conservation list for those species
slist = sensitivelist['name']
slistloc = conservationlist.loc[conservationlist['name'].isin(slist)]
slistloc

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,class,order,family,...,scientificNameAuthorship,status,npwActStatusComments,epbcStatus,epbcActStatusComments,subspeciesAuthor,majorGroup,subSpecies,sa_geoprivacy,co_status
130,4032000,Elanus scriptus,Letter-winged Kite,Elanus scriptus,https://biodiversity.org.au/afd/taxa/4a6524fb-...,dr653,"[{'key': 'class', 'value': 'AVES'}, {'key': 'o...",AVES,ACCIPITRIFORMES,ACCIPITRIDAE,...,"Gould, 1842",Vulnerable,Changed from R to V 1/Jan/2020.,NaN,NaN,NaN,NaN,NaN,open,Vulnerable
131,4032490,Haliaeetus leucogaster,White-bellied Sea-eagle,Haliaeetus (Pontoaetus) leucogaster,https://biodiversity.org.au/afd/taxa/dc420306-...,dr653,"[{'key': 'class', 'value': 'AVES'}, {'key': 'o...",AVES,ACCIPITRIFORMES,ACCIPITRIDAE,...,"(J.F. Gmelin, 1788)",Endangered,NaN,NaN,NaN,NaN,NaN,NaN,open,Endangered
134,4032081,Lophoictinia isura,Square-tailed Kite,Lophoictinia isura,https://biodiversity.org.au/afd/taxa/715a2874-...,dr653,"[{'key': 'class', 'value': 'AVES'}, {'key': 'o...",AVES,ACCIPITRIFORMES,ACCIPITRIDAE,...,"(Gould, 1838)",Endangered,NaN,NaN,NaN,NaN,NaN,NaN,open,Endangered
148,4032089,Thinornis cucullatus cucullatus,Eastern Hooded Dotteral,Thinornis cucullatus cucullatus,https://biodiversity.org.au/afd/taxa/a1591e76-...,dr653,"[{'key': 'class', 'value': 'AVES'}, {'key': 'o...",AVES,CHARADRIIFORMES,CHARADRIIDAE,...,"(J.F. Gmelin, 1789)",Vulnerable,NaN,VU,listed 6/11/2014,NaN,NaN,NaN,open,Vulnerable
181,4032696,Falco hypoleucos,Grey Falcon,Falco (Hierofalco) hypoleucos,https://biodiversity.org.au/afd/taxa/4c73a934-...,dr653,"[{'key': 'class', 'value': 'AVES'}, {'key': 'o...",AVES,FALCONIFORMES,FALCONIDAE,...,"Gould, 1841",Rare,NaN,VU,listed July 2020,NaN,NaN,NaN,open,Rare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,4032315,Thelymitra inflata,Inflated Sun-orchid,Thelymitra inflata,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra inflata Jeanes,Vulnerable,"VU A2a;C2a(i), updated null to V, Apr 2007",NaN,NaN,NaN,Angiospermae,NaN,open,Vulnerable
1165,4032765,Thelymitra latifolia,Blue Star Sun-orchid,Thelymitra latifolia,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra latifolia R.J.Bates,Vulnerable,Warning: Conservation status data auto-copied ...,NaN,NaN,NaN,Angiospermae,NaN,open,Vulnerable
1166,4031963,Thelymitra malvina,Mauvetuft Sun-orchid,Thelymitra malvina,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,"Thelymitra malvina M.A.Clem., D.L.Jones & Molloy",Endangered,NaN,NaN,NaN,NaN,Angiospermae,NaN,open,Endangered
1167,4032775,Thelymitra matthewsii,Spiral Sun-orchid,Thelymitra matthewsii,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra matthewsii Cheeseman,Endangered,NaN,VU,2000-07-16 00:00:00,NaN,Angiospermae,NaN,open,Endangered


In [81]:
mlist=  sensitivelist.merge(slistloc[['status','name']],how="left",left_on="name",right_on="name")
mlist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,Grouping,SPECIESTYPE,taxonID,vernacular name,Clarifier,Threaten a species/environment;,Threaten someones privacy;,Reason,sa_geoprivacy,status_x,status_y
0,24233,Tiliqua adelaidensis,Pygmy Bluetongue,Tiliqua adelaidensis,https://biodiversity.org.au/afd/taxa/a70c19ab-...,dr884,"[{'key': 'Grouping', 'value': 'Reptiles'}, {'k...",Reptiles,R,Y02572,Pygmy Blue-tongue Lizard (listed as Pygmy Blue...,All known location and burrow records,y,y,The Pygmy Blue-tongue Lizard is endemic to SA ...,obscured,sensitive,NaN
1,24377,Calyptorhynchus banksii graptogyne,Red-tailed Black Cockatoo (south-east),Calyptorhynchus (Calyptorhynchus) banksii grap...,https://biodiversity.org.au/afd/taxa/f8ecc7e3-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,Y09312,Red-tailed Black-Cockatoo (south east subspecies),All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive,Endangered
2,24336,Calyptorhynchus lathami halmaturinus,Glossy Black-cockatoo (kangaroo Island Subspec...,Calyptorhynchus (Calyptorhynchus) lathami halm...,https://biodiversity.org.au/afd/taxa/d2f5121c-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,C00265,Glossy Black-Cockatoo (Kangaroo Island subspec...,All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive,NaN
3,24331,Lophochroa leadbeateri,Major Mitchell's Cockatoo,Lophochroa leadbeateri,https://biodiversity.org.au/afd/taxa/5815e99d-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,U00270,Major Mitchell's Cockatoo (listed as Cacatua l...,All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive,NaN
4,24267,Calyptorhynchus funereus,Yellow-tailed Black-cockatoo,Zanda funerea,https://biodiversity.org.au/afd/taxa/145b081d-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,G00267,Yellow-tailed Black Cockatoo,All known nest hollows and artificial nest boxes.,y,NaN,some associated subspecies records are sensitive,obscured,sensitive,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,4221188,Ixodia achillaeoides subsp. arenicola,NaN,Ixodia achillaeoides subsp. arenicola,https://id.biodiversity.org.au/node/apni/2916083,dr884,"[{'key': 'Grouping', 'value': 'Garden Trade -O...",Garden Trade -Other Ornamental Horticulture pl...,P,ALA concept,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive,NaN
247,4221189,Asplenium bulbiferum subsp. gracillimum,Mother Spleenwort,Asplenium bulbiferum subsp. gracillimum,https://id.biodiversity.org.au/node/apni/2910577,dr884,"[{'key': 'Grouping', 'value': 'Ferns & Allies'...",Ferns & Allies,P,ALA concept,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,Taxon is listed as ssp. gracillimum so subsp. ...,obscured,sensitive,NaN
248,4221190,Asplenium bulbiferum,Hen And Chickens,Asplenium bulbiferum,https://id.biodiversity.org.au/node/apni/2887052,dr884,"[{'key': 'Grouping', 'value': 'Ferns & Allies'...",Ferns & Allies,P,NaN,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive,NaN
249,4221191,Lagenophora sublyrata,Slender Bottle-daisy,Lagenophora sublyrata,https://id.biodiversity.org.au/name/apni/51338482,dr884,"[{'key': 'Grouping', 'value': 'Garden Trade -O...",Garden Trade -Other Ornamental Horticulture pl...,P,na,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive,Vulnerable


In [82]:
sensitivelist['status']  = mlist.status_y.fillna(mlist.status_x)
sensitivelist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,Grouping,SPECIESTYPE,taxonID,vernacular name,Clarifier,Threaten a species/environment;,Threaten someones privacy;,Reason,sa_geoprivacy,status
0,24233,Tiliqua adelaidensis,Pygmy Bluetongue,Tiliqua adelaidensis,https://biodiversity.org.au/afd/taxa/a70c19ab-...,dr884,"[{'key': 'Grouping', 'value': 'Reptiles'}, {'k...",Reptiles,R,Y02572,Pygmy Blue-tongue Lizard (listed as Pygmy Blue...,All known location and burrow records,y,y,The Pygmy Blue-tongue Lizard is endemic to SA ...,obscured,sensitive
1,24377,Calyptorhynchus banksii graptogyne,Red-tailed Black Cockatoo (south-east),Calyptorhynchus (Calyptorhynchus) banksii grap...,https://biodiversity.org.au/afd/taxa/f8ecc7e3-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,Y09312,Red-tailed Black-Cockatoo (south east subspecies),All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,Endangered
2,24336,Calyptorhynchus lathami halmaturinus,Glossy Black-cockatoo (kangaroo Island Subspec...,Calyptorhynchus (Calyptorhynchus) lathami halm...,https://biodiversity.org.au/afd/taxa/d2f5121c-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,C00265,Glossy Black-Cockatoo (Kangaroo Island subspec...,All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive
3,24331,Lophochroa leadbeateri,Major Mitchell's Cockatoo,Lophochroa leadbeateri,https://biodiversity.org.au/afd/taxa/5815e99d-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,U00270,Major Mitchell's Cockatoo (listed as Cacatua l...,All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive
4,24267,Calyptorhynchus funereus,Yellow-tailed Black-cockatoo,Zanda funerea,https://biodiversity.org.au/afd/taxa/145b081d-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,G00267,Yellow-tailed Black Cockatoo,All known nest hollows and artificial nest boxes.,y,NaN,some associated subspecies records are sensitive,obscured,sensitive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,4221188,Ixodia achillaeoides subsp. arenicola,NaN,Ixodia achillaeoides subsp. arenicola,https://id.biodiversity.org.au/node/apni/2916083,dr884,"[{'key': 'Grouping', 'value': 'Garden Trade -O...",Garden Trade -Other Ornamental Horticulture pl...,P,ALA concept,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive
247,4221189,Asplenium bulbiferum subsp. gracillimum,Mother Spleenwort,Asplenium bulbiferum subsp. gracillimum,https://id.biodiversity.org.au/node/apni/2910577,dr884,"[{'key': 'Grouping', 'value': 'Ferns & Allies'...",Ferns & Allies,P,ALA concept,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,Taxon is listed as ssp. gracillimum so subsp. ...,obscured,sensitive
248,4221190,Asplenium bulbiferum,Hen And Chickens,Asplenium bulbiferum,https://id.biodiversity.org.au/node/apni/2887052,dr884,"[{'key': 'Grouping', 'value': 'Ferns & Allies'...",Ferns & Allies,P,NaN,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive
249,4221191,Lagenophora sublyrata,Slender Bottle-daisy,Lagenophora sublyrata,https://id.biodiversity.org.au/name/apni/51338482,dr884,"[{'key': 'Grouping', 'value': 'Garden Trade -O...",Garden Trade -Other Ornamental Horticulture pl...,P,na,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,Vulnerable


In [83]:
conservationlist = pd.read_csv(sourcedir + "sa-ala-conservation.csv", dtype=str)
conservationlist['sa_geoprivacy'] = 'open'
conservationlist['co_status'] = conservationlist['status']
conservationlist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,class,order,family,...,scientificNameAuthorship,status,npwActStatusComments,epbcStatus,epbcActStatusComments,subspeciesAuthor,majorGroup,subSpecies,sa_geoprivacy,co_status
0,4032241,Ornithorhynchus anatinus,Platypus,Ornithorhynchus anatinus,https://biodiversity.org.au/afd/taxa/ac61fd14-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,MONOTREMATA,ORNITHORHYNCHIDAE,...,"(Shaw, 1799)",Endangered,NaN,NaN,NaN,NaN,NaN,NaN,open,Endangered
1,4032333,Tachyglossus aculeatus multiaculeatus,Short-beaked Echidna (kangaroo Island),Tachyglossus aculeatus multiaculeatus,https://biodiversity.org.au/afd/taxa/e9289876-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,MONOTREMATA,TACHYGLOSSIDAE,...,"(Rothschild, 1905)",Endangered,Listing official as off 1/Jan/2020.,EN,NaN,NaN,NaN,NaN,open,Endangered
2,4031684,Myrmecobius fasciatus,Numbat,Myrmecobius fasciatus,https://biodiversity.org.au/afd/taxa/6c72d199-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,DASYUROMORPHIA,MYRMECOBIIDAE,...,"Waterhouse, 1836",Endangered,Introduced,EN,Introduced to South Australia. Changed from VU...,NaN,NaN,NaN,open,Endangered
3,4032310,Dasycercus blythi,Brush-tailed Mulgara,Dasycercus blythi,https://biodiversity.org.au/afd/taxa/926b1c33-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,DASYUROMORPHIA,DASYURIDAE,...,"(Krefft, 1867)",Endangered,NaN,NaN,NaN,NaN,NaN,NaN,open,Endangered
4,4031916,Dasyuroides byrnei,Kowari,Dasyuroides byrnei,https://biodiversity.org.au/afd/taxa/c342ff42-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,DASYUROMORPHIA,DASYURIDAE,...,"(Spencer, 1896)",Endangered,Changed from V to E 1/Jan/2020.,VU,NaN,NaN,NaN,NaN,open,Endangered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,4032775,Thelymitra matthewsii,Spiral Sun-orchid,Thelymitra matthewsii,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra matthewsii Cheeseman,Endangered,NaN,VU,2000-07-16 00:00:00,NaN,Angiospermae,NaN,open,Endangered
1168,4032248,Thelymitra occidentalis,Western Azure Sun Orchid,Thelymitra occidentalis,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra occidentalis Jeanes,Rare,RA d(i),NaN,NaN,NaN,Angiospermae,NaN,open,Rare
1169,4031900,Thelymitra orientalis,Plum Sun-orchid,Thelymitra orientalis,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra orientalis R.J.Bates,Rare,NaN,NaN,NaN,NaN,Angiospermae,NaN,open,Rare
1170,4032428,Thelymitra peniculata,Blue Star Sun-orchid,Thelymitra peniculata,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra peniculata Jeanes,Vulnerable,Warning: Conservation status data auto-copied ...,NaN,NaN,NaN,Angiospermae,NaN,open,Vulnerable


In [84]:
# join them in a way that works for inat (eg sensitive list, geoprivacy = 'obscured'
statelist = pd.concat([sensitivelist[[ 'taxonID', 'name',  'status', 'sa_geoprivacy', 'lsid']],
                       conservationlist[[ 'taxonID', 'name',  'status', 'sa_geoprivacy', 'lsid']]]).drop_duplicates()
# retrieve binomial and trinomial names from GBIF
parsednames = lf.gbifparse(statelist)
parsednames.to_csv(sourcedir + "sa-gbif.csv", index=False)
statelist = statelist.merge(parsednames[['scientificName','canonicalName','canonicalNameComplete','type','rankMarker']],how="left",left_on="name",right_on="scientificName")
numfullstatelist = len(statelist.index)
statelist = statelist.rename(columns={'taxonID':'sa_taxonID', 'name':'sa_name','status':'sa_status'})
statelist['sa_scientificName'] = statelist['canonicalName']
statelist

,sa_taxonID,sa_name,sa_status,sa_geoprivacy,lsid,scientificName,canonicalName,canonicalNameComplete,type,rankMarker,sa_scientificName
0,Y02572,Tiliqua adelaidensis,sensitive,obscured,https://biodiversity.org.au/afd/taxa/a70c19ab-...,Tiliqua adelaidensis,Tiliqua adelaidensis,Tiliqua adelaidensis,SCIENTIFIC,sp.,Tiliqua adelaidensis
1,Y09312,Calyptorhynchus banksii graptogyne,Endangered,obscured,https://biodiversity.org.au/afd/taxa/f8ecc7e3-...,Calyptorhynchus banksii graptogyne,Calyptorhynchus banksii graptogyne,Calyptorhynchus banksii graptogyne,SCIENTIFIC,infrasp.,Calyptorhynchus banksii graptogyne
2,Y09312,Calyptorhynchus banksii graptogyne,Endangered,obscured,https://biodiversity.org.au/afd/taxa/f8ecc7e3-...,Calyptorhynchus banksii graptogyne,Calyptorhynchus banksii graptogyne,Calyptorhynchus banksii graptogyne,SCIENTIFIC,infrasp.,Calyptorhynchus banksii graptogyne
3,C00265,Calyptorhynchus lathami halmaturinus,sensitive,obscured,https://biodiversity.org.au/afd/taxa/d2f5121c-...,Calyptorhynchus lathami halmaturinus,Calyptorhynchus lathami halmaturinus,Calyptorhynchus lathami halmaturinus,SCIENTIFIC,infrasp.,Calyptorhynchus lathami halmaturinus
4,U00270,Lophochroa leadbeateri,sensitive,obscured,https://biodiversity.org.au/afd/taxa/5815e99d-...,Lophochroa leadbeateri,Lophochroa leadbeateri,Lophochroa leadbeateri,SCIENTIFIC,sp.,Lophochroa leadbeateri
...,...,...,...,...,...,...,...,...,...,...,...
1769,S06033,Thelymitra occidentalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,Thelymitra occidentalis,Thelymitra occidentalis,Thelymitra occidentalis,SCIENTIFIC,sp.,Thelymitra occidentalis
1770,K32213,Thelymitra orientalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,Thelymitra orientalis,Thelymitra orientalis,Thelymitra orientalis,SCIENTIFIC,sp.,Thelymitra orientalis
1771,S32209,Thelymitra peniculata,Vulnerable,open,https://id.biodiversity.org.au/taxon/apni/5141...,Thelymitra peniculata,Thelymitra peniculata,Thelymitra peniculata,SCIENTIFIC,sp.,Thelymitra peniculata
1772,S32209,Thelymitra peniculata,Vulnerable,open,https://id.biodiversity.org.au/taxon/apni/5141...,Thelymitra peniculata,Thelymitra peniculata,Thelymitra peniculata,SCIENTIFIC,sp.,Thelymitra peniculata


In [85]:
# Identify records that won't comply with iNaturalist species names
noncomply = statelist[statelist['type'].isin(['INFORMAL','CULTIVAR','HYBRID', 'BLACKLISTED']) ]
noncomply

,sa_taxonID,sa_name,sa_status,sa_geoprivacy,lsid,scientificName,canonicalName,canonicalNameComplete,type,rankMarker,sa_scientificName
195,E06266,Caladenia sp. Monarto South (H.Goldsack 163 AD...,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5139...,Caladenia sp. Monarto South (H.Goldsack 163 AD...,Caladenia sp.Monarto,Caladenia sp.Monarto,INFORMAL,sp.,Caladenia sp.Monarto
196,E06266,Caladenia sp. Monarto South (H.Goldsack 163 AD...,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5139...,Caladenia sp. Monarto South (H.Goldsack 163 AD...,Caladenia sp.Monarto,Caladenia sp.Monarto,INFORMAL,sp.,Caladenia sp.Monarto
235,S06113,Eriochilus sp. Swamp (D.E.Murfet 1950b),sensitive,obscured,https://id.biodiversity.org.au/taxon/apni/5129...,Eriochilus sp. Swamp (D.E.Murfet 1950b),Eriochilus sp.Swamp(D.E.Murfet-1950b),Eriochilus sp.Swamp(D.E.Murfet-1950b),INFORMAL,sp.,Eriochilus sp.Swamp(D.E.Murfet-1950b)
266,Y06292,Prasophyllum sp. Enigma (R.Bates 2350),sensitive,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,Prasophyllum sp. Enigma (R.Bates 2350),Prasophyllum spec.,Prasophyllum spec. Enigma,INFORMAL,sp.,Prasophyllum spec.
267,Q06288,Prasophyllum sp. Waterholes (R.Bates 9037),Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,Prasophyllum sp. Waterholes (R.Bates 9037),Prasophyllum spec.,Prasophyllum spec. Waterholes,INFORMAL,sp.,Prasophyllum spec.
268,Q06288,Prasophyllum sp. Waterholes (R.Bates 9037),Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,Prasophyllum sp. Waterholes (R.Bates 9037),Prasophyllum spec.,Prasophyllum spec. Waterholes,INFORMAL,sp.,Prasophyllum spec.
305,W05463,Pterostylis sp. Hale (R.Bates 21725),Vulnerable,obscured,https://id.biodiversity.org.au/taxon/apni/5141...,Pterostylis sp. Hale (R.Bates 21725),Pterostylis spec.,Pterostylis spec. Hale,INFORMAL,sp.,Pterostylis spec.
306,W05463,Pterostylis sp. Hale (R.Bates 21725),Vulnerable,obscured,https://id.biodiversity.org.au/taxon/apni/5141...,Pterostylis sp. Hale (R.Bates 21725),Pterostylis spec.,Pterostylis spec. Hale,INFORMAL,sp.,Pterostylis spec.
307,G04403,"Pterostylis sp. Rock ledges (pl. 185, Bates & ...",Endangered,obscured,https://id.biodiversity.org.au/name/apni/194722,"Pterostylis sp. Rock ledges (pl. 185, Bates & ...",Pterostylis ledges sp.Rock,Pterostylis ledges sp. sp.Rock,INFORMAL,sp.,Pterostylis ledges sp.Rock
308,G04403,"Pterostylis sp. Rock ledges (pl. 185, Bates & ...",Endangered,obscured,https://id.biodiversity.org.au/name/apni/194722,"Pterostylis sp. Rock ledges (pl. 185, Bates & ...",Pterostylis ledges sp.Rock,Pterostylis ledges sp. sp.Rock,INFORMAL,sp.,Pterostylis ledges sp.Rock


In [86]:
# remove records that do not comply
statelist = statelist[~statelist['type'].isin(['INFORMAL','CULTIVAR','HYBRID', 'BLACKLISTED']) ]
statelist = pd.DataFrame(statelist[['sa_scientificName','sa_status','sa_geoprivacy','lsid']]).drop_duplicates()
statelist

,sa_scientificName,sa_status,sa_geoprivacy,lsid
0,Tiliqua adelaidensis,sensitive,obscured,https://biodiversity.org.au/afd/taxa/a70c19ab-...
1,Calyptorhynchus banksii graptogyne,Endangered,obscured,https://biodiversity.org.au/afd/taxa/f8ecc7e3-...
3,Calyptorhynchus lathami halmaturinus,sensitive,obscured,https://biodiversity.org.au/afd/taxa/d2f5121c-...
4,Lophochroa leadbeateri,sensitive,obscured,https://biodiversity.org.au/afd/taxa/5815e99d-...
5,Calyptorhynchus funereus,sensitive,obscured,https://biodiversity.org.au/afd/taxa/145b081d-...
...,...,...,...,...
1767,Thelymitra matthewsii,Endangered,open,https://id.biodiversity.org.au/taxon/apni/5141...
1769,Thelymitra occidentalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...
1770,Thelymitra orientalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...
1771,Thelymitra peniculata,Vulnerable,open,https://id.biodiversity.org.au/taxon/apni/5141...


In [87]:
#horrible fudge to get this through tonight
statelist.at[0, 'sa_geoprivacy'] = 'obscured'
statelist.at[1, 'sa_geoprivacy'] = 'obscured'
statelist.at[3, 'sa_geoprivacy'] = 'obscured'
statelist.at[4, 'sa_geoprivacy'] = 'obscured'
statelist.at[7, 'sa_geoprivacy'] = 'obscured'
statelist.at[8, 'sa_geoprivacy'] = 'obscured'
statelist.at[10, 'sa_geoprivacy'] = 'obscured'
statelist.at[12, 'sa_geoprivacy'] = 'obscured'
statelist.at[13, 'sa_geoprivacy'] = 'obscured'
statelist.at[15, 'sa_geoprivacy'] = 'obscured'
statelist.at[17, 'sa_geoprivacy'] = 'obscured'
statelist.at[20, 'sa_geoprivacy'] = 'obscured'
statelist.at[23, 'sa_geoprivacy'] = 'obscured'
statelist.at[26, 'sa_geoprivacy'] = 'obscured'
statelist.at[28, 'sa_geoprivacy'] = 'obscured'
statelist.at[30, 'sa_geoprivacy'] = 'obscured'
statelist.at[32, 'sa_geoprivacy'] = 'obscured'
statelist.at[34, 'sa_geoprivacy'] = 'obscured'
statelist.at[36, 'sa_geoprivacy'] = 'obscured'
statelist.at[38, 'sa_geoprivacy'] = 'obscured'
statelist.at[40, 'sa_geoprivacy'] = 'obscured'
statelist.at[42, 'sa_geoprivacy'] = 'obscured'
statelist.at[44, 'sa_geoprivacy'] = 'obscured'
statelist.at[46, 'sa_geoprivacy'] = 'obscured'
statelist.at[48, 'sa_geoprivacy'] = 'obscured'
statelist.at[50, 'sa_geoprivacy'] = 'obscured'
statelist.at[52, 'sa_geoprivacy'] = 'obscured'
statelist.at[54, 'sa_geoprivacy'] = 'obscured'
statelist.at[56, 'sa_geoprivacy'] = 'obscured'
statelist.at[58, 'sa_geoprivacy'] = 'obscured'
statelist.at[60, 'sa_geoprivacy'] = 'obscured'
statelist.at[62, 'sa_geoprivacy'] = 'obscured'
statelist.at[64, 'sa_geoprivacy'] = 'obscured'
statelist.at[66, 'sa_geoprivacy'] = 'obscured'
statelist.at[68, 'sa_geoprivacy'] = 'obscured'
statelist.at[70, 'sa_geoprivacy'] = 'obscured'
statelist.at[72, 'sa_geoprivacy'] = 'obscured'
statelist.at[74, 'sa_geoprivacy'] = 'obscured'
statelist.at[76, 'sa_geoprivacy'] = 'obscured'
statelist.at[78, 'sa_geoprivacy'] = 'obscured'
statelist.at[80, 'sa_geoprivacy'] = 'obscured'
statelist.at[82, 'sa_geoprivacy'] = 'obscured'
statelist.at[85, 'sa_geoprivacy'] = 'obscured'
statelist.at[87, 'sa_geoprivacy'] = 'obscured'
statelist.at[89, 'sa_geoprivacy'] = 'obscured'
statelist.at[91, 'sa_geoprivacy'] = 'obscured'
statelist.at[93, 'sa_geoprivacy'] = 'obscured'
statelist.at[95, 'sa_geoprivacy'] = 'obscured'
statelist.at[97, 'sa_geoprivacy'] = 'obscured'
statelist.at[99, 'sa_geoprivacy'] = 'obscured'
statelist.at[102, 'sa_geoprivacy'] = 'obscured'
statelist.at[104, 'sa_geoprivacy'] = 'obscured'
statelist.at[106, 'sa_geoprivacy'] = 'obscured'
statelist.at[108, 'sa_geoprivacy'] = 'obscured'
statelist.at[110, 'sa_geoprivacy'] = 'obscured'
statelist.at[112, 'sa_geoprivacy'] = 'obscured'
statelist.at[114, 'sa_geoprivacy'] = 'obscured'
statelist.at[116, 'sa_geoprivacy'] = 'obscured'
statelist.at[118, 'sa_geoprivacy'] = 'obscured'
statelist.at[120, 'sa_geoprivacy'] = 'obscured'
statelist.at[122, 'sa_geoprivacy'] = 'obscured'
statelist.at[124, 'sa_geoprivacy'] = 'obscured'
statelist.at[126, 'sa_geoprivacy'] = 'obscured'
statelist.at[128, 'sa_geoprivacy'] = 'obscured'
statelist.at[130, 'sa_geoprivacy'] = 'obscured'
statelist.at[132, 'sa_geoprivacy'] = 'obscured'
statelist.at[134, 'sa_geoprivacy'] = 'obscured'
statelist.at[136, 'sa_geoprivacy'] = 'obscured'
statelist.at[138, 'sa_geoprivacy'] = 'obscured'
statelist.at[140, 'sa_geoprivacy'] = 'obscured'
statelist.at[142, 'sa_geoprivacy'] = 'obscured'
statelist.at[144, 'sa_geoprivacy'] = 'obscured'
statelist.at[146, 'sa_geoprivacy'] = 'obscured'
statelist.at[148, 'sa_geoprivacy'] = 'obscured'
statelist.at[150, 'sa_geoprivacy'] = 'obscured'
statelist.at[152, 'sa_geoprivacy'] = 'obscured'
statelist.at[154, 'sa_geoprivacy'] = 'obscured'
statelist.at[156, 'sa_geoprivacy'] = 'obscured'
statelist.at[158, 'sa_geoprivacy'] = 'obscured'
statelist.at[160, 'sa_geoprivacy'] = 'obscured'
statelist.at[162, 'sa_geoprivacy'] = 'obscured'
statelist.at[164, 'sa_geoprivacy'] = 'obscured'
statelist.at[166, 'sa_geoprivacy'] = 'obscured'
statelist.at[168, 'sa_geoprivacy'] = 'obscured'
statelist.at[170, 'sa_geoprivacy'] = 'obscured'
statelist.at[172, 'sa_geoprivacy'] = 'obscured'
statelist.at[175, 'sa_geoprivacy'] = 'obscured'
statelist.at[177, 'sa_geoprivacy'] = 'obscured'
statelist.at[180, 'sa_geoprivacy'] = 'obscured'
statelist.at[183, 'sa_geoprivacy'] = 'obscured'
statelist.at[185, 'sa_geoprivacy'] = 'obscured'
statelist.at[187, 'sa_geoprivacy'] = 'obscured'
statelist.at[189, 'sa_geoprivacy'] = 'obscured'
statelist.at[191, 'sa_geoprivacy'] = 'obscured'
statelist.at[193, 'sa_geoprivacy'] = 'obscured'
statelist.at[198, 'sa_geoprivacy'] = 'obscured'
statelist.at[200, 'sa_geoprivacy'] = 'obscured'
statelist.at[202, 'sa_geoprivacy'] = 'obscured'
statelist.at[204, 'sa_geoprivacy'] = 'obscured'
statelist.at[206, 'sa_geoprivacy'] = 'obscured'
statelist.at[208, 'sa_geoprivacy'] = 'obscured'
statelist.at[210, 'sa_geoprivacy'] = 'obscured'
statelist.at[212, 'sa_geoprivacy'] = 'obscured'
statelist.at[214, 'sa_geoprivacy'] = 'obscured'
statelist.at[216, 'sa_geoprivacy'] = 'obscured'
statelist.at[217, 'sa_geoprivacy'] = 'obscured'
statelist.at[219, 'sa_geoprivacy'] = 'obscured'
statelist.at[221, 'sa_geoprivacy'] = 'obscured'
statelist.at[223, 'sa_geoprivacy'] = 'obscured'
statelist.at[225, 'sa_geoprivacy'] = 'obscured'
statelist.at[227, 'sa_geoprivacy'] = 'obscured'
statelist.at[229, 'sa_geoprivacy'] = 'obscured'
statelist.at[231, 'sa_geoprivacy'] = 'obscured'
statelist.at[233, 'sa_geoprivacy'] = 'obscured'
statelist.at[236, 'sa_geoprivacy'] = 'obscured'
statelist.at[238, 'sa_geoprivacy'] = 'obscured'
statelist.at[240, 'sa_geoprivacy'] = 'obscured'
statelist.at[242, 'sa_geoprivacy'] = 'obscured'
statelist.at[244, 'sa_geoprivacy'] = 'obscured'
statelist.at[246, 'sa_geoprivacy'] = 'obscured'
statelist.at[248, 'sa_geoprivacy'] = 'obscured'
statelist.at[250, 'sa_geoprivacy'] = 'obscured'
statelist.at[252, 'sa_geoprivacy'] = 'obscured'
statelist.at[254, 'sa_geoprivacy'] = 'obscured'
statelist.at[256, 'sa_geoprivacy'] = 'obscured'
statelist.at[258, 'sa_geoprivacy'] = 'obscured'
statelist.at[260, 'sa_geoprivacy'] = 'obscured'
statelist.at[262, 'sa_geoprivacy'] = 'obscured'
statelist.at[264, 'sa_geoprivacy'] = 'obscured'
statelist.at[269, 'sa_geoprivacy'] = 'obscured'
statelist.at[271, 'sa_geoprivacy'] = 'obscured'
statelist.at[273, 'sa_geoprivacy'] = 'obscured'
statelist.at[275, 'sa_geoprivacy'] = 'obscured'
statelist.at[277, 'sa_geoprivacy'] = 'obscured'
statelist.at[279, 'sa_geoprivacy'] = 'obscured'
statelist.at[281, 'sa_geoprivacy'] = 'obscured'
statelist.at[283, 'sa_geoprivacy'] = 'obscured'
statelist.at[285, 'sa_geoprivacy'] = 'obscured'
statelist.at[287, 'sa_geoprivacy'] = 'obscured'
statelist.at[289, 'sa_geoprivacy'] = 'obscured'
statelist.at[291, 'sa_geoprivacy'] = 'obscured'
statelist.at[293, 'sa_geoprivacy'] = 'obscured'
statelist.at[295, 'sa_geoprivacy'] = 'obscured'
statelist.at[297, 'sa_geoprivacy'] = 'obscured'
statelist.at[299, 'sa_geoprivacy'] = 'obscured'
statelist.at[301, 'sa_geoprivacy'] = 'obscured'
statelist.at[303, 'sa_geoprivacy'] = 'obscured'
statelist.at[310, 'sa_geoprivacy'] = 'obscured'
statelist.at[312, 'sa_geoprivacy'] = 'obscured'
statelist.at[314, 'sa_geoprivacy'] = 'obscured'
statelist.at[316, 'sa_geoprivacy'] = 'obscured'
statelist.at[318, 'sa_geoprivacy'] = 'obscured'
statelist.at[320, 'sa_geoprivacy'] = 'obscured'
statelist.at[322, 'sa_geoprivacy'] = 'obscured'
statelist.at[324, 'sa_geoprivacy'] = 'obscured'
statelist.at[326, 'sa_geoprivacy'] = 'obscured'
statelist.at[328, 'sa_geoprivacy'] = 'obscured'
statelist.at[330, 'sa_geoprivacy'] = 'obscured'
statelist.at[332, 'sa_geoprivacy'] = 'obscured'
statelist.at[334, 'sa_geoprivacy'] = 'obscured'
statelist.at[336, 'sa_geoprivacy'] = 'obscured'
statelist.at[338, 'sa_geoprivacy'] = 'obscured'
statelist.at[340, 'sa_geoprivacy'] = 'obscured'
statelist.at[344, 'sa_geoprivacy'] = 'obscured'
statelist.at[349, 'sa_geoprivacy'] = 'obscured'
statelist.at[354, 'sa_geoprivacy'] = 'obscured'
statelist.at[357, 'sa_geoprivacy'] = 'obscured'
statelist.at[365, 'sa_geoprivacy'] = 'obscured'
statelist.at[367, 'sa_geoprivacy'] = 'obscured'
statelist.at[370, 'sa_geoprivacy'] = 'obscured'
statelist.at[377, 'sa_geoprivacy'] = 'obscured'
statelist.at[379, 'sa_geoprivacy'] = 'obscured'
statelist.at[384, 'sa_geoprivacy'] = 'obscured'
statelist.at[386, 'sa_geoprivacy'] = 'obscured'
statelist.at[388, 'sa_geoprivacy'] = 'obscured'
statelist.at[392, 'sa_geoprivacy'] = 'obscured'
statelist.at[394, 'sa_geoprivacy'] = 'obscured'
statelist.at[396, 'sa_geoprivacy'] = 'obscured'
statelist.at[413, 'sa_geoprivacy'] = 'obscured'
statelist.at[414, 'sa_geoprivacy'] = 'obscured'
statelist.at[415, 'sa_geoprivacy'] = 'obscured'
statelist.at[418, 'sa_geoprivacy'] = 'obscured'
statelist.at[419, 'sa_geoprivacy'] = 'obscured'
statelist.at[421, 'sa_geoprivacy'] = 'obscured'
statelist.at[423, 'sa_geoprivacy'] = 'obscured'
statelist.at[555, 'sa_geoprivacy'] = 'obscured'
statelist.at[557, 'sa_geoprivacy'] = 'obscured'
statelist.at[561, 'sa_geoprivacy'] = 'obscured'
statelist.at[576, 'sa_geoprivacy'] = 'obscured'
statelist.at[610, 'sa_geoprivacy'] = 'obscured'
statelist.at[612, 'sa_geoprivacy'] = 'obscured'
statelist.at[615, 'sa_geoprivacy'] = 'obscured'
statelist.at[617, 'sa_geoprivacy'] = 'obscured'
statelist.at[618, 'sa_geoprivacy'] = 'obscured'
statelist.at[622, 'sa_geoprivacy'] = 'obscured'
statelist.at[627, 'sa_geoprivacy'] = 'obscured'
statelist.at[632, 'sa_geoprivacy'] = 'obscured'
statelist.at[634, 'sa_geoprivacy'] = 'obscured'
statelist.at[642, 'sa_geoprivacy'] = 'obscured'
statelist.at[645, 'sa_geoprivacy'] = 'obscured'
statelist.at[647, 'sa_geoprivacy'] = 'obscured'
statelist.at[692, 'sa_geoprivacy'] = 'obscured'
statelist.at[712, 'sa_geoprivacy'] = 'obscured'
statelist.at[743, 'sa_geoprivacy'] = 'obscured'
statelist.at[771, 'sa_geoprivacy'] = 'obscured'
statelist.at[774, 'sa_geoprivacy'] = 'obscured'
statelist.at[776, 'sa_geoprivacy'] = 'obscured'
statelist.at[783, 'sa_geoprivacy'] = 'obscured'
statelist.at[785, 'sa_geoprivacy'] = 'obscured'
statelist.at[787, 'sa_geoprivacy'] = 'obscured'
statelist.at[789, 'sa_geoprivacy'] = 'obscured'
statelist.at[791, 'sa_geoprivacy'] = 'obscured'
statelist.at[793, 'sa_geoprivacy'] = 'obscured'
statelist.at[796, 'sa_geoprivacy'] = 'obscured'
statelist.at[800, 'sa_geoprivacy'] = 'obscured'
statelist.at[803, 'sa_geoprivacy'] = 'obscured'
statelist.at[806, 'sa_geoprivacy'] = 'obscured'
statelist.at[808, 'sa_geoprivacy'] = 'obscured'
statelist.at[810, 'sa_geoprivacy'] = 'obscured'
statelist.at[835, 'sa_geoprivacy'] = 'obscured'
statelist.at[844, 'sa_geoprivacy'] = 'obscured'
statelist.at[997, 'sa_geoprivacy'] = 'obscured'
statelist.at[999, 'sa_geoprivacy'] = 'obscured'
statelist.at[1002, 'sa_geoprivacy'] = 'obscured'
statelist.at[1006, 'sa_geoprivacy'] = 'obscured'
statelist.at[1008, 'sa_geoprivacy'] = 'obscured'
statelist.at[1011, 'sa_geoprivacy'] = 'obscured'
statelist.at[1013, 'sa_geoprivacy'] = 'obscured'
statelist.at[1016, 'sa_geoprivacy'] = 'obscured'
statelist.at[1018, 'sa_geoprivacy'] = 'obscured'
statelist.at[1049, 'sa_geoprivacy'] = 'obscured'
statelist.at[1088, 'sa_geoprivacy'] = 'obscured'
statelist.at[1168, 'sa_geoprivacy'] = 'obscured'
statelist.at[1171, 'sa_geoprivacy'] = 'obscured'
statelist.at[1173, 'sa_geoprivacy'] = 'obscured'
statelist.at[1183, 'sa_geoprivacy'] = 'obscured'
statelist.at[1190, 'sa_geoprivacy'] = 'obscured'
statelist.at[1232, 'sa_geoprivacy'] = 'obscured'
statelist.at[1234, 'sa_geoprivacy'] = 'obscured'
statelist.at[1251, 'sa_geoprivacy'] = 'obscured'
statelist.at[1259, 'sa_geoprivacy'] = 'obscured'
statelist.at[1273, 'sa_geoprivacy'] = 'obscured'
statelist.at[1275, 'sa_geoprivacy'] = 'obscured'
statelist.at[1278, 'sa_geoprivacy'] = 'obscured'
statelist.at[1295, 'sa_geoprivacy'] = 'obscured'
statelist.at[1298, 'sa_geoprivacy'] = 'obscured'
statelist.at[1299, 'sa_geoprivacy'] = 'obscured'
statelist.at[1306, 'sa_geoprivacy'] = 'obscured'
statelist.at[1314, 'sa_geoprivacy'] = 'obscured'
statelist.at[1316, 'sa_geoprivacy'] = 'obscured'
statelist.at[1324, 'sa_geoprivacy'] = 'obscured'
statelist.at[1326, 'sa_geoprivacy'] = 'obscured'
statelist.at[1331, 'sa_geoprivacy'] = 'obscured'
statelist.at[1337, 'sa_geoprivacy'] = 'obscured'
statelist.at[1356, 'sa_geoprivacy'] = 'obscured'
statelist.at[1358, 'sa_geoprivacy'] = 'obscured'
statelist.at[1360, 'sa_geoprivacy'] = 'obscured'
statelist.at[1362, 'sa_geoprivacy'] = 'obscured'
statelist.at[1364, 'sa_geoprivacy'] = 'obscured'
statelist.at[1381, 'sa_geoprivacy'] = 'obscured'
statelist.at[1386, 'sa_geoprivacy'] = 'obscured'
statelist.at[1388, 'sa_geoprivacy'] = 'obscured'
statelist.at[1393, 'sa_geoprivacy'] = 'obscured'
statelist.at[1396, 'sa_geoprivacy'] = 'obscured'
statelist.at[1398, 'sa_geoprivacy'] = 'obscured'
statelist.at[1402, 'sa_geoprivacy'] = 'obscured'
statelist.at[1404, 'sa_geoprivacy'] = 'obscured'
statelist.at[1406, 'sa_geoprivacy'] = 'obscured'
statelist.at[1411, 'sa_geoprivacy'] = 'obscured'
statelist.at[1413, 'sa_geoprivacy'] = 'obscured'
statelist.at[1418, 'sa_geoprivacy'] = 'obscured'
statelist.at[1426, 'sa_geoprivacy'] = 'obscured'
statelist.at[1532, 'sa_geoprivacy'] = 'obscured'
statelist.at[1534, 'sa_geoprivacy'] = 'obscured'
statelist.at[1538, 'sa_geoprivacy'] = 'obscured'
statelist.at[1540, 'sa_geoprivacy'] = 'obscured'
statelist.at[1542, 'sa_geoprivacy'] = 'obscured'
statelist.at[1545, 'sa_geoprivacy'] = 'obscured'
statelist.at[1547, 'sa_geoprivacy'] = 'obscured'
statelist.at[1552, 'sa_geoprivacy'] = 'obscured'
statelist.at[1554, 'sa_geoprivacy'] = 'obscured'
statelist.at[1556, 'sa_geoprivacy'] = 'obscured'
statelist.at[1558, 'sa_geoprivacy'] = 'obscured'
statelist.at[1560, 'sa_geoprivacy'] = 'obscured'
statelist.at[1562, 'sa_geoprivacy'] = 'obscured'
statelist.at[1564, 'sa_geoprivacy'] = 'obscured'
statelist.at[1569, 'sa_geoprivacy'] = 'obscured'
statelist.at[1571, 'sa_geoprivacy'] = 'obscured'
statelist.at[1574, 'sa_geoprivacy'] = 'obscured'
statelist.at[1576, 'sa_geoprivacy'] = 'obscured'
statelist.at[1578, 'sa_geoprivacy'] = 'obscured'
statelist.at[1582, 'sa_geoprivacy'] = 'obscured'
statelist.at[1584, 'sa_geoprivacy'] = 'obscured'
statelist.at[1592, 'sa_geoprivacy'] = 'obscured'
statelist.at[1594, 'sa_geoprivacy'] = 'obscured'
statelist.at[1598, 'sa_geoprivacy'] = 'obscured'
statelist.at[1601, 'sa_geoprivacy'] = 'obscured'
statelist.at[1604, 'sa_geoprivacy'] = 'obscured'
statelist.at[1606, 'sa_geoprivacy'] = 'obscured'
statelist.at[1609, 'sa_geoprivacy'] = 'obscured'
statelist.at[1612, 'sa_geoprivacy'] = 'obscured'
statelist.at[1614, 'sa_geoprivacy'] = 'obscured'
statelist.at[1616, 'sa_geoprivacy'] = 'obscured'
statelist.at[1618, 'sa_geoprivacy'] = 'obscured'
statelist.at[1621, 'sa_geoprivacy'] = 'obscured'
statelist.at[1623, 'sa_geoprivacy'] = 'obscured'
statelist.at[1625, 'sa_geoprivacy'] = 'obscured'
statelist.at[1627, 'sa_geoprivacy'] = 'obscured'
statelist.at[1629, 'sa_geoprivacy'] = 'obscured'
statelist.at[1631, 'sa_geoprivacy'] = 'obscured'
statelist.at[1633, 'sa_geoprivacy'] = 'obscured'
statelist.at[1635, 'sa_geoprivacy'] = 'obscured'
statelist.at[1641, 'sa_geoprivacy'] = 'obscured'
statelist.at[1643, 'sa_geoprivacy'] = 'obscured'
statelist.at[1645, 'sa_geoprivacy'] = 'obscured'
statelist.at[1647, 'sa_geoprivacy'] = 'obscured'
statelist.at[1650, 'sa_geoprivacy'] = 'obscured'
statelist.at[1653, 'sa_geoprivacy'] = 'obscured'
statelist.at[1656, 'sa_geoprivacy'] = 'obscured'
statelist.at[1658, 'sa_geoprivacy'] = 'obscured'
statelist.at[1661, 'sa_geoprivacy'] = 'obscured'
statelist.at[1663, 'sa_geoprivacy'] = 'obscured'
statelist.at[1667, 'sa_geoprivacy'] = 'obscured'
statelist.at[1669, 'sa_geoprivacy'] = 'obscured'
statelist.at[1671, 'sa_geoprivacy'] = 'obscured'
statelist.at[1674, 'sa_geoprivacy'] = 'obscured'
statelist.at[1677, 'sa_geoprivacy'] = 'obscured'
statelist.at[1679, 'sa_geoprivacy'] = 'obscured'
statelist.at[1684, 'sa_geoprivacy'] = 'obscured'
statelist.at[1687, 'sa_geoprivacy'] = 'obscured'
statelist.at[1689, 'sa_geoprivacy'] = 'obscured'
statelist.at[1691, 'sa_geoprivacy'] = 'obscured'
statelist.at[1693, 'sa_geoprivacy'] = 'obscured'
statelist.at[1695, 'sa_geoprivacy'] = 'obscured'
statelist.at[1698, 'sa_geoprivacy'] = 'obscured'
statelist.at[1700, 'sa_geoprivacy'] = 'obscured'
statelist.at[1703, 'sa_geoprivacy'] = 'obscured'
statelist.at[1705, 'sa_geoprivacy'] = 'obscured'
statelist.at[1707, 'sa_geoprivacy'] = 'obscured'
statelist.at[1710, 'sa_geoprivacy'] = 'obscured'
statelist.at[1712, 'sa_geoprivacy'] = 'obscured'
statelist.at[1714, 'sa_geoprivacy'] = 'obscured'
statelist.at[1717, 'sa_geoprivacy'] = 'obscured'
statelist.at[1719, 'sa_geoprivacy'] = 'obscured'
statelist.at[1721, 'sa_geoprivacy'] = 'obscured'
statelist.at[1723, 'sa_geoprivacy'] = 'obscured'
statelist.at[1725, 'sa_geoprivacy'] = 'obscured'
statelist.at[1732, 'sa_geoprivacy'] = 'obscured'
statelist.at[1734, 'sa_geoprivacy'] = 'obscured'
statelist.at[1736, 'sa_geoprivacy'] = 'obscured'
statelist.at[1738, 'sa_geoprivacy'] = 'obscured'
statelist.at[1744, 'sa_geoprivacy'] = 'obscured'
statelist.at[1746, 'sa_geoprivacy'] = 'obscured'
statelist.at[1748, 'sa_geoprivacy'] = 'obscured'
statelist.at[1750, 'sa_geoprivacy'] = 'obscured'
statelist.at[1752, 'sa_geoprivacy'] = 'obscured'
statelist.at[1756, 'sa_geoprivacy'] = 'obscured'
statelist.at[1758, 'sa_geoprivacy'] = 'obscured'
statelist.at[1760, 'sa_geoprivacy'] = 'obscured'
statelist.at[1763, 'sa_geoprivacy'] = 'obscured'
statelist.at[1765, 'sa_geoprivacy'] = 'obscured'
statelist.at[1767, 'sa_geoprivacy'] = 'obscured'
statelist.at[1771, 'sa_geoprivacy'] = 'obscured'
statelist = statelist.drop_duplicates()
statelist.at[216, 'sa_status'] = 'Endangered'
statelist.at[418, 'sa_status'] = 'Endangered'
statelist.at[419, 'sa_status'] = 'Endangered'
statelist = statelist.drop([3, 4, 7, 12, 58, 99, 414, 423], axis=0)
statelist

,sa_scientificName,sa_status,sa_geoprivacy,lsid
0,Tiliqua adelaidensis,sensitive,obscured,https://biodiversity.org.au/afd/taxa/a70c19ab-...
1,Calyptorhynchus banksii graptogyne,Endangered,obscured,https://biodiversity.org.au/afd/taxa/f8ecc7e3-...
5,Calyptorhynchus funereus,sensitive,obscured,https://biodiversity.org.au/afd/taxa/145b081d-...
6,Euastacus bispinosa,sensitive,obscured,https://biodiversity.org.au/afd/taxa/fa41771a-...
8,Polytelis anthopeplus monarchoides,Vulnerable,obscured,https://biodiversity.org.au/afd/taxa/1958a1ff-...
...,...,...,...,...
1755,Thelymitra grandiflora,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...
1762,Thelymitra ixioides,Endangered,open,https://id.biodiversity.org.au/taxon/apni/5141...
1769,Thelymitra occidentalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...
1770,Thelymitra orientalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...


In [88]:
parsednames['type'].unique()

array(['SCIENTIFIC', 'INFORMAL'], dtype=object)

In [89]:
dupinformation = statelist.groupby('sa_scientificName').filter(lambda x: len(x) > 1)#.sort('size',ascending=False)
dupinformation
# Really odd that these are still appearing. They do clean up by the end.
# I had to manually delete about 3 records from the final csv file.

,sa_scientificName,sa_status,sa_geoprivacy,lsid
0,Tiliqua adelaidensis,sensitive,obscured,https://biodiversity.org.au/afd/taxa/a70c19ab-...
52,Wurmbea dioica lacunaria,Endangered,obscured,ALA_DR653_914
146,Thysanotus tuberosus parviflorus,Vulnerable,obscured,ALA_DR653_909
148,Thysanotus tuberosus tuberosus,Vulnerable,obscured,https://id.biodiversity.org.au/node/apni/2889586
152,Xanthorrhoea minor lutea,Endangered,obscured,ALA_DR653_921
281,Pterostylis cucullata cucullata,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5141...
283,Pterostylis cucullata sylvicola,Endangered,obscured,ALA_DR653_1129
344,Pterostylis cucullata sylvicola,sensitive,obscured,https://id.biodiversity.org.au/taxon/apni/5141...
345,Pterostylis cucullata cucullata,sensitive,obscured,https://id.biodiversity.org.au/taxon/apni/5141...
349,Wurmbea dioica lacunaria,sensitive,obscured,https://id.biodiversity.org.au/node/apni/2902098


### 4. Equivalent IUCN statuses

In [90]:
# check for duplicates with conflicting information
iucn_statuses = {'Not Evaluated', 'Data Deficient', 'Least Concern', 'Near Threatened', 'Vulnerable', 'Endangered', 'Critically Endangered', 'Extinct in the Wild','Extinct'}
statelist.groupby(['sa_status'])['sa_status'].count()

sa_status
Endangered    299
Rare          605
Vulnerable    252
sensitive      57
Name: sa_status, dtype: int64

In [91]:
# these will be used to populate the iucn_equivalent field
iucnStatusMappings = {
    'rare': 'Rare',
    'endangered': 'Endangered',
    'vulnerable':'Vulnerable',
    'sensitive': 'Vulnerable'
}

### 5. Determine best place ID to use

In [92]:
inatstatuses.groupby(['place_id','place_name','place_display_name'])['place_id'].count()
# looks like 6827 - note for extract


place_id  place_name                      place_display_name            
124968    South Australia, marine waters  South Australia, marine waters      2
6899      South Australia                 South Australia, AU               124
Name: place_id, dtype: int64

## Merge iNaturalist statuses with State lists on scientificName

1. Match - updates, even if the statuses are the same we'll update the links and values anyway
2. No match - statuses to be added (additions)
   1.1 No match and no taxnomy - search for synonyms
   1.2 No match
3. Merge the other direction to see if there are deletes?


In [93]:
# join to see which lists already have a status in inaturalist based on scientificName
mergedstatuses = statelist[['sa_scientificName','sa_status','sa_geoprivacy','lsid']].merge(inatstatuses[['status_id','scientificName','taxon_id','user_id','description','iucn','authority','status','geoprivacy','place_id','place_display_name']],how="left",left_on='sa_scientificName',right_on='scientificName',suffixes=(None,'_inat')).sort_values(['scientificName'])
mergedstatuses


,sa_scientificName,sa_status,sa_geoprivacy,lsid,status_id,scientificName,taxon_id,user_id,description,iucn,authority,status,geoprivacy,place_id,place_display_name
29,Adiantum capillus-veneris,Vulnerable,obscured,https://id.biodiversity.org.au/node/apni/2887223,152329,Adiantum capillus-veneris,63916,708886,,40,SA DEWNR,endangered,obscured,6899,"South Australia, AU"
554,Anogramma leptophylla,Rare,open,https://id.biodiversity.org.au/taxon/apni/5129...,162866,Anogramma leptophylla,180646,702203,,20,"National Parks and Wildlife Act, 1972",Rare,,6899,"South Australia, AU"
40,Arthropodium milleflorum,Vulnerable,obscured,https://id.biodiversity.org.au/node/apni/2902983,152354,Arthropodium milleflorum,522233,708886,,40,SA DEWNR,endangered,obscured,6899,"South Australia, AU"
30,Blechnum chambersii,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5126...,152381,Blechnum chambersii,323506,708886,,40,SA DEWNR,endangered,obscured,6899,"South Australia, AU"
41,Brachyscome diversifolia,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5126...,152402,Brachyscome diversifolia,520376,708886,,40,SA DEWNR,endangered,obscured,6899,"South Australia, AU"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,Thelymitra grandiflora,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1209,Thelymitra ixioides,Endangered,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1210,Thelymitra occidentalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1211,Thelymitra orientalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# prepare the export fields, common to New template and Update template
# new statuses
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username
# url is either a florabase url or bie page
biesearchurl = "https://bie.ala.org.au/species/" # eg + "https://id.biodiversity.org.au/node/apni/2894366"
mergedstatuses['new_url'] =  biesearchurl + mergedstatuses['lsid']
# biesearchurl = "https://bie.ala.org.au/species/" # eg + "https://id.biodiversity.org.au/node/apni/2894366"
mergedstatuses['new_description'] = "Listed as Threatened - refer to https://www.environment.sa.gov.au/topics/plants-and-animals/threatened-species-and-ecological-communities/threatened-species"
mergedstatuses['new_authority'] = "South Australia Department for Environment and Water  https://www.environment.sa.gov.au"
mergedstatuses.rename(columns={'sa_geoprivacy':'new_geoprivacy'},inplace=True)
mergedstatuses['new_place_id'] = '7830'  # Victoria, AU
mergedstatuses['new_username'] = 'peggydnew'
mergedstatuses['new_iucn_equivalent'] = mergedstatuses['status'].str.lower().str.strip().map(iucnStatusMappings).fillna('Vulnerable') # map to dictionary
mergedstatuses['new_status'] = mergedstatuses['sa_status'].fillna('Sensitive')
mergedstatuses

,sa_scientificName,sa_status,new_geoprivacy,lsid,status_id,scientificName,taxon_id,user_id,description,iucn,...,geoprivacy,place_id,place_display_name,new_url,new_description,new_authority,new_place_id,new_username,new_iucn_equivalent,new_status
29,Adiantum capillus-veneris,Vulnerable,obscured,https://id.biodiversity.org.au/node/apni/2887223,152329,Adiantum capillus-veneris,63916,708886,,40,...,obscured,6899,"South Australia, AU",https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,7830,peggydnew,Endangered,Vulnerable
554,Anogramma leptophylla,Rare,open,https://id.biodiversity.org.au/taxon/apni/5129...,162866,Anogramma leptophylla,180646,702203,,20,...,,6899,"South Australia, AU",https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,7830,peggydnew,Rare,Rare
40,Arthropodium milleflorum,Vulnerable,obscured,https://id.biodiversity.org.au/node/apni/2902983,152354,Arthropodium milleflorum,522233,708886,,40,...,obscured,6899,"South Australia, AU",https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,7830,peggydnew,Endangered,Vulnerable
30,Blechnum chambersii,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5126...,152381,Blechnum chambersii,323506,708886,,40,...,obscured,6899,"South Australia, AU",https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,7830,peggydnew,Endangered,Endangered
41,Brachyscome diversifolia,Endangered,obscured,https://id.biodiversity.org.au/taxon/apni/5126...,152402,Brachyscome diversifolia,520376,708886,,40,...,obscured,6899,"South Australia, AU",https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,7830,peggydnew,Endangered,Endangered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,Thelymitra grandiflora,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,7830,peggydnew,Vulnerable,Rare
1209,Thelymitra ixioides,Endangered,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,7830,peggydnew,Vulnerable,Endangered
1210,Thelymitra occidentalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,7830,peggydnew,Vulnerable,Rare
1211,Thelymitra orientalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,7830,peggydnew,Vulnerable,Rare


## Updates

In [95]:
# those that need to be updated - we found a status
mergedstatuses[mergedstatuses['status_id'].notnull()][['sa_scientificName','sa_status','status_id','taxon_id','status','new_geoprivacy','geoprivacy','authority','user_id']]

,sa_scientificName,sa_status,status_id,taxon_id,status,new_geoprivacy,geoprivacy,authority,user_id
29,Adiantum capillus-veneris,Vulnerable,152329,63916,endangered,obscured,obscured,SA DEWNR,708886
554,Anogramma leptophylla,Rare,162866,180646,Rare,open,,"National Parks and Wildlife Act, 1972",702203
40,Arthropodium milleflorum,Vulnerable,152354,522233,endangered,obscured,obscured,SA DEWNR,708886
30,Blechnum chambersii,Endangered,152381,323506,endangered,obscured,obscured,SA DEWNR,708886
41,Brachyscome diversifolia,Endangered,152402,520376,endangered,obscured,obscured,SA DEWNR,708886
...,...,...,...,...,...,...,...,...,...
519,Tiliqua adelaidensis,Endangered,152348,37472,endangered,obscured,obscured,SA DEWNR,708886
0,Tiliqua adelaidensis,sensitive,152348,37472,endangered,obscured,obscured,SA DEWNR,708886
39,Todea barbara,Endangered,152368,323949,endangered,obscured,obscured,SA DEWNR,708886
24,Utricularia beaugleholei,Vulnerable,152394,508359,endangered,obscured,obscured,SA DEWNR,708886


In [96]:
# updates - create the data frame
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
updates = pd.DataFrame(mergedstatuses[mergedstatuses['status_id'].notnull()])
updates.sort_values('scientificName')
updates['action'] = 'UPDATE'
#updates.loc[:,'action'] = 'UPDATE'
updates = updates[['action','scientificName','status_id','taxon_id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
updates.columns = updates.columns.str.replace("new_", "", regex=True)
updates = updates.rename(columns={'scientificName':'taxon_name',
                                  'status_id':'id'})
updates

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
29,UPDATE,Adiantum capillus-veneris,152329,63916,Vulnerable,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,7830,peggydnew,Listed as Threatened - refer to https://www.en...
554,UPDATE,Anogramma leptophylla,162866,180646,Rare,Rare,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,7830,peggydnew,Listed as Threatened - refer to https://www.en...
40,UPDATE,Arthropodium milleflorum,152354,522233,Vulnerable,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,7830,peggydnew,Listed as Threatened - refer to https://www.en...
30,UPDATE,Blechnum chambersii,152381,323506,Endangered,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,7830,peggydnew,Listed as Threatened - refer to https://www.en...
41,UPDATE,Brachyscome diversifolia,152402,520376,Endangered,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,7830,peggydnew,Listed as Threatened - refer to https://www.en...
...,...,...,...,...,...,...,...,...,...,...,...,...
519,UPDATE,Tiliqua adelaidensis,152348,37472,Endangered,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://biodive...,obscured,7830,peggydnew,Listed as Threatened - refer to https://www.en...
0,UPDATE,Tiliqua adelaidensis,152348,37472,sensitive,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://biodive...,obscured,7830,peggydnew,Listed as Threatened - refer to https://www.en...
39,UPDATE,Todea barbara,152368,323949,Endangered,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,7830,peggydnew,Listed as Threatened - refer to https://www.en...
24,UPDATE,Utricularia beaugleholei,152394,508359,Vulnerable,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,7830,peggydnew,Listed as Threatened - refer to https://www.en...


## No status in iNaturalist via straight scientificName match
The SA records that didn't match up to a status in iNaturalist

In [97]:
# to add: those that have no inaturalist status - 532!!
noinatstatus = mergedstatuses[mergedstatuses['status_id'].isnull()]
# try to match the taxon name to something in inaturalist
noinatstatus = noinatstatus.merge(inattaxa, how="left", left_on="sa_scientificName",right_on="scientificName")
noinatstatus

,sa_scientificName,sa_status,new_geoprivacy,lsid,status_id,scientificName_x,taxon_id,user_id,description,iucn,...,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName_y,taxonRank,references
0,Calyptorhynchus banksii graptogyne,Endangered,obscured,https://biodiversity.org.au/afd/taxa/f8ecc7e3-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Aves,Psittaciformes,Cacatuidae,Calyptorhynchus,banksii,graptogyne,2021-09-16T20:01:13Z,Calyptorhynchus banksii graptogyne,subspecies,NaN
1,Calyptorhynchus funereus,sensitive,obscured,https://biodiversity.org.au/afd/taxa/145b081d-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Euastacus bispinosa,sensitive,obscured,https://biodiversity.org.au/afd/taxa/fa41771a-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Falco hypoleucos,Rare,obscured,https://biodiversity.org.au/afd/taxa/4c73a934-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Aves,Falconiformes,Falconidae,Falco,hypoleucos,NaN,2019-11-23T01:02:07Z,Falco hypoleucos,species,http://www.birdlife.org/datazone/speciesfactsh...
4,Falco peregrinus,sensitive,obscured,https://biodiversity.org.au/afd/taxa/083b413f-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Aves,Falconiformes,Falconidae,Falco,peregrinus,NaN,2022-04-03T07:11:44Z,Falco peregrinus,species,http://www.birdlife.org/datazone/speciesfactsh...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,Thelymitra grandiflora,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,Liliopsida,Asparagales,Orchidaceae,Thelymitra,grandiflora,NaN,2019-02-12T18:48:26Z,Thelymitra grandiflora,species,http://www.catalogueoflife.org/annual-checklis...
1100,Thelymitra ixioides,Endangered,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,Liliopsida,Asparagales,Orchidaceae,Thelymitra,ixioides,NaN,2020-11-05T03:12:47Z,Thelymitra ixioides,species,http://www.catalogueoflife.org/annual-checklis...
1101,Thelymitra occidentalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,Liliopsida,Asparagales,Orchidaceae,Thelymitra,occidentalis,NaN,2019-02-12T18:48:43Z,Thelymitra occidentalis,species,http://www.catalogueoflife.org/annual-checklis...
1102,Thelymitra orientalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,Liliopsida,Asparagales,Orchidaceae,Thelymitra,orientalis,NaN,2021-07-28T03:47:32Z,Thelymitra orientalis,species,https://eol.org/pages/47162334


In [98]:
noinatstatus[noinatstatus['id'].notna()] # there's no status but there is a matching inat taxon (id is the taxon id)
# note: "Dendrobium" matches to both genus and section

,sa_scientificName,sa_status,new_geoprivacy,lsid,status_id,scientificName_x,taxon_id,user_id,description,iucn,...,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName_y,taxonRank,references
0,Calyptorhynchus banksii graptogyne,Endangered,obscured,https://biodiversity.org.au/afd/taxa/f8ecc7e3-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Aves,Psittaciformes,Cacatuidae,Calyptorhynchus,banksii,graptogyne,2021-09-16T20:01:13Z,Calyptorhynchus banksii graptogyne,subspecies,NaN
3,Falco hypoleucos,Rare,obscured,https://biodiversity.org.au/afd/taxa/4c73a934-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Aves,Falconiformes,Falconidae,Falco,hypoleucos,NaN,2019-11-23T01:02:07Z,Falco hypoleucos,species,http://www.birdlife.org/datazone/speciesfactsh...
4,Falco peregrinus,sensitive,obscured,https://biodiversity.org.au/afd/taxa/083b413f-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Aves,Falconiformes,Falconidae,Falco,peregrinus,NaN,2022-04-03T07:11:44Z,Falco peregrinus,species,http://www.birdlife.org/datazone/speciesfactsh...
5,Haliaeetus leucogaster,Endangered,obscured,https://biodiversity.org.au/afd/taxa/dc420306-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Aves,Accipitriformes,Accipitridae,Haliaeetus,leucogaster,NaN,2022-05-07T18:58:43Z,Haliaeetus leucogaster,species,http://www.birdlife.org/datazone/speciesfactsh...
8,Montia fontana chondrosperma,Vulnerable,obscured,ALA_DR653_390,NaN,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Caryophyllales,Montiaceae,Montia,fontana,chondrosperma,2014-10-13T01:18:24Z,Montia fontana chondrosperma,subspecies,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,Thelymitra flexuosa,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,Liliopsida,Asparagales,Orchidaceae,Thelymitra,flexuosa,NaN,2019-02-12T18:48:22Z,Thelymitra flexuosa,species,http://www.catalogueoflife.org/annual-checklis...
1099,Thelymitra grandiflora,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,Liliopsida,Asparagales,Orchidaceae,Thelymitra,grandiflora,NaN,2019-02-12T18:48:26Z,Thelymitra grandiflora,species,http://www.catalogueoflife.org/annual-checklis...
1100,Thelymitra ixioides,Endangered,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,Liliopsida,Asparagales,Orchidaceae,Thelymitra,ixioides,NaN,2020-11-05T03:12:47Z,Thelymitra ixioides,species,http://www.catalogueoflife.org/annual-checklis...
1101,Thelymitra occidentalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,Liliopsida,Asparagales,Orchidaceae,Thelymitra,occidentalis,NaN,2019-02-12T18:48:43Z,Thelymitra occidentalis,species,http://www.catalogueoflife.org/annual-checklis...


In [99]:
# there's no status but there is a matching inat taxon (id is the taxon id)
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions

,sa_scientificName,sa_status,new_geoprivacy,lsid,status_id,scientificName_x,taxon_id,user_id,description,iucn,...,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName_y,taxonRank,references
0,Calyptorhynchus banksii graptogyne,Endangered,obscured,https://biodiversity.org.au/afd/taxa/f8ecc7e3-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Aves,Psittaciformes,Cacatuidae,Calyptorhynchus,banksii,graptogyne,2021-09-16T20:01:13Z,Calyptorhynchus banksii graptogyne,subspecies,NaN
3,Falco hypoleucos,Rare,obscured,https://biodiversity.org.au/afd/taxa/4c73a934-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Aves,Falconiformes,Falconidae,Falco,hypoleucos,NaN,2019-11-23T01:02:07Z,Falco hypoleucos,species,http://www.birdlife.org/datazone/speciesfactsh...
4,Falco peregrinus,sensitive,obscured,https://biodiversity.org.au/afd/taxa/083b413f-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Aves,Falconiformes,Falconidae,Falco,peregrinus,NaN,2022-04-03T07:11:44Z,Falco peregrinus,species,http://www.birdlife.org/datazone/speciesfactsh...
5,Haliaeetus leucogaster,Endangered,obscured,https://biodiversity.org.au/afd/taxa/dc420306-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Aves,Accipitriformes,Accipitridae,Haliaeetus,leucogaster,NaN,2022-05-07T18:58:43Z,Haliaeetus leucogaster,species,http://www.birdlife.org/datazone/speciesfactsh...
8,Montia fontana chondrosperma,Vulnerable,obscured,ALA_DR653_390,NaN,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Caryophyllales,Montiaceae,Montia,fontana,chondrosperma,2014-10-13T01:18:24Z,Montia fontana chondrosperma,subspecies,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,Thelymitra flexuosa,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,Liliopsida,Asparagales,Orchidaceae,Thelymitra,flexuosa,NaN,2019-02-12T18:48:22Z,Thelymitra flexuosa,species,http://www.catalogueoflife.org/annual-checklis...
1099,Thelymitra grandiflora,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,Liliopsida,Asparagales,Orchidaceae,Thelymitra,grandiflora,NaN,2019-02-12T18:48:26Z,Thelymitra grandiflora,species,http://www.catalogueoflife.org/annual-checklis...
1100,Thelymitra ixioides,Endangered,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,Liliopsida,Asparagales,Orchidaceae,Thelymitra,ixioides,NaN,2020-11-05T03:12:47Z,Thelymitra ixioides,species,http://www.catalogueoflife.org/annual-checklis...
1101,Thelymitra occidentalis,Rare,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,Liliopsida,Asparagales,Orchidaceae,Thelymitra,occidentalis,NaN,2019-02-12T18:48:43Z,Thelymitra occidentalis,species,http://www.catalogueoflife.org/annual-checklis...


In [100]:
additions['scientificName'] = additions['sa_scientificName']
#additions['new_status'] = additions['wa_status']
additions.sort_values(['scientificName'])
additions['action'] = 'ADD'
additions = additions[['action','scientificName','status_id','id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
additions.columns = additions.columns.str.replace("new_", "", regex=True)
additions = additions.rename(columns={'scientificName':'taxon_name',
                                      'id':'taxon_id',
                                  'status_id':'id'})
additions

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
0,ADD,Calyptorhynchus banksii graptogyne,NaN,720261,Endangered,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://biodive...,obscured,7830,peggydnew,Listed as Threatened - refer to https://www.en...
3,ADD,Falco hypoleucos,NaN,4642,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://biodive...,obscured,7830,peggydnew,Listed as Threatened - refer to https://www.en...
4,ADD,Falco peregrinus,NaN,4647,sensitive,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://biodive...,obscured,7830,peggydnew,Listed as Threatened - refer to https://www.en...
5,ADD,Haliaeetus leucogaster,NaN,5315,Endangered,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://biodive...,obscured,7830,peggydnew,Listed as Threatened - refer to https://www.en...
8,ADD,Montia fontana chondrosperma,NaN,412835,Vulnerable,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/ALA_DR653_390,obscured,7830,peggydnew,Listed as Threatened - refer to https://www.en...
...,...,...,...,...,...,...,...,...,...,...,...,...
1098,ADD,Thelymitra flexuosa,NaN,323933,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,7830,peggydnew,Listed as Threatened - refer to https://www.en...
1099,ADD,Thelymitra grandiflora,NaN,785816,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,7830,peggydnew,Listed as Threatened - refer to https://www.en...
1100,ADD,Thelymitra ixioides,NaN,83359,Endangered,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,7830,peggydnew,Listed as Threatened - refer to https://www.en...
1101,ADD,Thelymitra occidentalis,NaN,568429,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,7830,peggydnew,Listed as Threatened - refer to https://www.en...


In [101]:
# manually had to remove spurious duplicates from output file
all = pd.concat([updates,additions])
# all = pd.concat([updates,additions]).drop_duplicates(['taxon_id'], keep = 'first')
all.to_csv(sourcedir + "sa.csv", index=False )


In [102]:
# all = all.drop_duplicates(['taxon_id'])
# all

In [103]:
# what didnt match to a taxon?
unknownToInat = noinatstatus[noinatstatus['id'].isna()]
unknownToInat

,sa_scientificName,sa_status,new_geoprivacy,lsid,status_id,scientificName_x,taxon_id,user_id,description,iucn,...,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName_y,taxonRank,references
1,Calyptorhynchus funereus,sensitive,obscured,https://biodiversity.org.au/afd/taxa/145b081d-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Euastacus bispinosa,sensitive,obscured,https://biodiversity.org.au/afd/taxa/fa41771a-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Pandion cristatus,sensitive,obscured,https://biodiversity.org.au/afd/taxa/e61096e6-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Gentianella gunniana,Vulnerable,obscured,https://id.biodiversity.org.au/node/apni/2908181,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Wurmbea dioica lacunaria,Endangered,obscured,ALA_DR653_914,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047,Schoenus discifer,Rare,open,https://id.biodiversity.org.au/node/apni/2910693,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1051,Schoenus sculptus,Rare,open,https://id.biodiversity.org.au/node/apni/2903833,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1077,Eriochilus paludosus,Endangered,open,https://id.biodiversity.org.au/taxon/apni/5129...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1087,Prasophyllum rotundiflorum,Endangered,open,https://id.biodiversity.org.au/taxon/apni/5140...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
noinatstatus[noinatstatus['id'].isna()].groupby('sa_status').size()

sa_status
Endangered    39
Rare          79
Vulnerable    42
sensitive     20
dtype: int64

### are there any that need to be removed?
qld list count: 2517
qld inat statuses count: 653

updates to inat status: 570
additional inat status: 1355
qld statuses we can't find a taxon match for in iNaturalist: 606
total: 2531 (explainable via the various genus/section entries that we matched to in the taxonomy)

inat statuses left over: 653-570=83 that may need checking against the above

In [105]:
# inat statuses that aren't in added or updated
notaddedupdated = inatstatuses[~inatstatuses['taxon_id'].isin(updates['taxon_id'])]
notaddedupdated

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
147,152333,100616,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Euastacus,bispinosus,,2022-06-06T16:26:39Z,Euastacus bispinosus,species,http://www.iucnredlist.org/apps/redlist/details,,,
89,180750,1038978,624851,6899,,"Department of Environment and Water, Governmen...",Endangered,,Known from just a few areas and near human dis...,obscured,...,Hibbertia,hirsuta,,2021-08-31T12:37:27Z,Hibbertia hirsuta,species,,,,
931,165449,12647,702203,6899,,South Australia,EN,https://bie.ala.org.au/species/urn:lsid:biodiv...,,obscured,...,Epthianura,crocea,,2021-09-17T08:46:17Z,Epthianura crocea,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
415,223574,1300466,3669610,6899,,"National Parks and Wildlife Act, 1972",NT,https://www.legislation.sa.gov.au/LZ/C/A/NATIO...,,,...,Hydrocotyle,glochidiata,,2021-10-31T04:06:15Z,Hydrocotyle glochidiata,species,http://www.plantsoftheworldonline.org/taxon/ur...,,,
590,261603,1359456,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Brownseya,serpentina,,2022-02-17T04:32:12Z,Brownseya serpentina,species,,,,
2699,154407,19251,,6899,,SA DEWNR,Vulnerable,,,obscured,...,Polytelis,anthopeplus,,2022-06-11T01:21:17Z,Polytelis anthopeplus,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
217,170165,20166,702203,6899,,Atlas of Living Australia,NT,https://bie.ala.org.au/species/urn:lsid:biodiv...,,,...,Ninox,connivens,,2021-07-28T02:17:03Z,Ninox connivens,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
2408,152357,210157,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Viola,betonicifolia,,2022-11-16T04:06:10Z,Viola betonicifolia,species,http://www.ubio.org/browser/details.php?nameba...,,,
1233,152409,323439,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Pterostylis,furcata,,2019-02-12T17:26:58Z,Pterostylis furcata,species,http://www.catalogueoflife.org/annual-checklis...,,,
111,152389,400260,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Caleana,minor,,2021-06-17T09:18:56Z,Caleana minor,species,,,,


In [106]:
# Stats
numsensitive = len(sensitivelist.index)
numconservation = len(conservationlist.index)
numupdates  = len(updates.index)
numadditions  = len(additions.index)
numnoinatstatus = len(noinatstatus.index)
numunknownToInat = len(unknownToInat.index)
numnotaddedupdated = len(notaddedupdated.index)
numnoncomply = len(noncomply.index)
numcomply = len(statelist.index)
numdupinfo = len(dupinformation.index)
d = {'Sensitive': [numsensitive],
    'Conservation': [numconservation],
    'Statelist merge': [numfullstatelist],
    'Species iNat Comply' : [numcomply],
    'Species iNat non-Comply': [numnoncomply],
    'Duplicate Information': [numdupinfo],
    'Updates': [numupdates],
    'Additions': [numadditions],
    'Not added updated': [numnotaddedupdated],
    'No Inat Status': [numnoinatstatus],
    'Unknown to Inat': [numunknownToInat]}

statsdf = pd.DataFrame(data=d)
statsdf

,Sensitive,Conservation,Statelist merge,Species iNat Comply,Species iNat non-Comply,Duplicate Information,Updates,Additions,Not added updated,No Inat Status,Unknown to Inat
0,251,1172,1774,1213,37,14,109,924,18,1104,180
